##Importa e defina a função para fazer requests

In [11]:
import requests
import json
import datetime
#doc api
#https://docs.api.jikan.moe/#tag/anime/operation/getAnimeForum
def pesquisa_anime(nome_anime, limite=0):

  url = "https://api.jikan.moe/v4/anime?q=" + nome_anime + "&limit=1"
  # Criar a requisição GET
  response = requests.get(url)
  filtered_data = []

  # Verificar se a requisição foi bem-sucedida
  if response.status_code == 200:
    # Converter o conteúdo da resposta para JSON
    dados_resposta = response.json()

    # Iterar pelos itens na chave 'data'
    for item in dados_resposta['data']:
        # Obter o título do tipo "Default"
        default_title = next((title['title'] for title in item['titles'] if title['type'] == "Default"), "N/A")

        # Coletar os gêneros e temas em listas
        genres = [genre['name'] for genre in item['genres']]
        themes = [theme['name'] for theme in item['themes']]

        # Criar um dicionário com as informações desejadas
        filtered_info = {
            'title': default_title,
            'synopsis': item['synopsis'],
            'genres': genres,
            'themes': themes
        }

        # Adicionar o dicionário filtrado à lista
        filtered_data.append(filtered_info)

  return filtered_data

import requests
import json

def ultimos_animes():
  """
  Retorna todos os últimos animes lançados no ano atual utilizando paginação.

  Returns:
      list[dict]: Lista de dicionários contendo informações sobre os animes.
  """

  filtered_data = []
  page = 1

  data_atual = datetime.datetime.now()
  primeiro_dia_ano = data_atual.strftime("%Y-01-01")
  ultimo_dia_ano = data_atual.strftime("%Y-12-31")

  while True:
    url = f"https://api.jikan.moe/v4/anime?start_date={primeiro_dia_ano}&end_date={ultimo_dia_ano}&limit=25&sort=desc&page={page}"
    response = requests.get(url)

    if response.status_code == 200:
      dados_resposta = response.json()
      animes_pagina = dados_resposta["data"]

      for item in animes_pagina:
        default_title = next((title['title'] for title in item['titles'] if title['type'] == "Default"), "N/A")
        genres = [genre['name'] for genre in item['genres']]
        themes = [theme['name'] for theme in item['themes']]

        filtered_info = {
          'title': default_title,
          'synopsis': item['synopsis'],
          'genres': genres,
          'themes': themes
        }

        filtered_data.append(filtered_info)

      has_next_page = dados_resposta["pagination"]["has_next_page"]
      if not has_next_page:
        break
      page += 1

  return filtered_data

###Gera a lista de animes mais recentes


In [12]:
lista_animes_novos = ultimos_animes()
print(f"Total de animes encontrados: {len(lista_animes_novos)}")

Total de animes encontrados: 85


##Importa e configura o Google Gemini

In [22]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

from google.colab import userdata
gemini_api = userdata.get('gemini_api')

#Configurações iniciais
import google.generativeai as genai

genai.configure(api_key=gemini_api)

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

chat = model.start_chat(history=[])

response = chat.send_message("Estou de passando uma lista de animes, e por essa lista quero que selecione 10 animes que eu goste, essa é aliste de animes mais atual: " + json.dumps(lista_animes_novos) + " favor retornar o resultado com linguagem natural e adicionar o motivo pela escolha, mas para isso aguarde até eu enviar outra mensagem com a informações do animes que eu gosto.")
print(response.text)

**Animes que você pode gostar:**

* **Nozomanu Fushi no Boukensha** - Um aventureiro fracassado se torna um esqueleto e embarca em uma jornada para recuperar sua forma humana. Você gosta de animes de fantasia e aventura.
* **Isekai de Mofumofu Nadenade suru Tame ni Ganbattemasu.** - Uma mulher é reencarnada em outro mundo e pode acariciar todos os tipos de criaturas fofas. Você gosta de animes de isekai e animais fofos.
* **Sasaki to Pii-chan** - Um homem de 39 anos adota um pardal falante que lhe concede poderes mágicos. Você gosta de animes de comédia e fantasia.
* **Majo to Yajuu** - Uma caçadora de bruxas e um mago viajam pelo mundo resolvendo casos relacionados a magia. Você gosta de animes de ação e fantasia.
* **Shin no Nakama ja Nai to Yuusha no Party wo Oidasareta node, Henkyou de Slow Life suru Koto ni Shimashita 2nd** - Um homem é expulso de seu grupo de heróis e decide viver uma vida tranquila no campo. Você gosta de animes de aventura e fantasia.
* **Saijaku Tamer wa Gomi 

##Cria um loop de perguntas

In [24]:
prompt = ""
while prompt != "fim":
  prompt = input('Deixe sugerir outros animes com base em um que você goste?\n')

  if prompt == "fim":
    break

  #recupera os dados do anime solicitado
  meu_anime = pesquisa_anime(prompt)
  if len(json.dumps(meu_anime)) == 0 or json.dumps(meu_anime)== "[]":
      print("Não encontrei esse anime, favor informar outro!")
      break
  else:
    response = chat.send_message( "Eu gosto do anime: " + json.dumps(meu_anime) + " Sugira animes com base na listagem passada anteriormente!")
    print(response.text)



Deixe sugerir outros animes com base em um que você goste?
Ore dake Level Up na Ken
**Animes que você pode gostar, com base em Ore dake Level Up na Ken:**

* **Nozomanu Fushi no Boukensha** - Um aventureiro fracassado se torna um esqueleto e embarca em uma jornada para recuperar sua forma humana. Ambos os animes apresentam protagonistas que são inicialmente fracos, mas ganham poderes extraordinários.
* **Shin no Nakama ja Nai to Yuusha no Party wo Oidasareta node, Henkyou de Slow Life suru Koto ni Shimashita 2nd** - Um homem é expulso de seu grupo de heróis e decide viver uma vida tranquila no campo. Ambos os animes apresentam protagonistas que são subestimados, mas possuem habilidades ocultas.
* **Saijaku Tamer wa Gomi Hiroi no Tabi wo Hajimemashita.** - Uma jovem com zero habilidades de domador de monstros é expulsa de sua aldeia e embarca em uma jornada para encontrar seu destino. Ambos os animes apresentam protagonistas que são considerados fracos, mas têm o potencial de se tornare